Importowanie bibliotek

In [ ]:
!pip install pandas numpy statsmodels

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy as scp
import seaborn as sns
import statsmodels.api as sm
from statsmodels.graphics.gofplots import qqplot
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.stats.diagnostic as smd
from statsmodels.stats.stattools import durbin_watson

Zadanie 1

In [ ]:
df_original = pd.read_csv("dane_projekt1.csv", sep=";")
df_original.info()

In [ ]:
liczba_obserwacji = len(df_original)
braki_danych = df_original.isnull().sum()
print(f"Liczba obserwacji: {liczba_obserwacji}\nBraki danych:\n{braki_danych}")

Usuwamy niepoprawne obserwacje.
Na przykład ujemne wydatki nie mają żadnego sensu.

In [ ]:
df = df_original.copy()

# Filtrowanie danych i usuwanie obserwacji z ujemnymi wydatkami
df = df[df['wydatki_zyw'] >= 0]

# Wyświetlenie informacji o nowej ramce danych
print("Liczba obserwacji po usunięciu błędnych:", len(df))

Zmienne ilościowe (7):  waga, wzrost, dzieci, wiek, dochód, oszczędności, wydatki_zyw


Zmienne jakościowe (3): płeć, jednoos, miejsce


Nie ma brakujących danych.

In [ ]:
# Obliczanie tabeli częstości dla zmiennej jakościowej 'plec'
tabela_czestosci_plec = df['plec'].value_counts()
print("Tabela częstości dla zmiennej 'plec':")
print(tabela_czestosci_plec)

# Obliczanie tabeli częstości dla zmiennej jakościowej 'jednoos'
tabela_czestosci_jednoos = df['jednoos'].value_counts()
print("\nTabela częstości dla zmiennej 'jednoos':")
print(tabela_czestosci_jednoos)

# Obliczanie tabeli częstości dla zmiennej jakościowej 'miejsce'
tabela_czestosci_miejsce = df['miejsce'].value_counts()
print("\nTabela częstości dla zmiennej 'miejsce':")
print(tabela_czestosci_miejsce)

Zmienne jakościowe:
płeć: jest niewielka dysproporcja między kobietami a mężczyznami
jednoos: jest mniej jenoosobowych niż wieloosobowych gospodarstw
miejsce: najwięcej danych jest z miejsca nr 2, z miejsc nr 1 i nr 3 jest mniej więcej tyle samo danych

In [ ]:
# Wyświetlenie statystyk opisowych dla zmiennych ilościowych
statystyka_opisowa = df[['waga', 'wzrost', 'dzieci', 'wiek', 'dochod', 'oszczednosci', 'wydatki_zyw']].describe()
print("Statystyki opisowe dla zmiennych ilościowych:")
print(statystyka_opisowa)

In [ ]:
import numpy as np
import scipy.stats as stats

# Tworzenie jednej figury i podwykresów
fig, axs = plt.subplots(2, 4, figsize=(15, 8))  # 2 wiersze, 4 kolumny

# Wykresy dla poszczególnych zmiennych ilościowych
variables = ['waga', 'wzrost', 'dzieci', 'wiek', 'dochod', 'oszczednosci', 'wydatki_zyw']
for i, var in enumerate(variables):
    ax = axs.flatten()[i]
    ax.hist(df[var], bins=20, alpha=0.5, color='blue', density=True)  # Histogram
    ax.set_title(f'Histogram {var}')
    ax.set_xlabel(var)
    ax.set_ylabel('Gęstość')

    # Wyznaczenie średniej i odchylenia standardowego
    mean = df[var].mean()
    std_dev = df[var].std()

    # Tworzenie danych dla rozkładu normalnego
    xmin, xmax = ax.get_xlim()
    x = np.linspace(xmin, xmax, 100)
    y = scp.stats.norm.pdf(x, mean, std_dev)

    # Narysowanie krzywej rozkładu normalnego
    ax.plot(x, y, color='red', linewidth=2)

# Ukrycie ostatniego podwykresu (jeśli jest)
axs[1, 3].axis('off')

# Wyświetlenie wykresów
fig.tight_layout()
fig.show()


Waga - podobny do normalnego.

Wzrost - podobny do normalnego

Dziecie - jeśli pamiętamy, że ta zmienna tak na prawdę nie jest zmienną ilościową, bo nie można mieć np 2.5 dziecka oraz nie zdaża się żeby mieć 100 dzieci, to można przyjąć, że rozkład jest podobny normalny.

Wiek - podobny do normalnego.

Dochód - widać zdecydowaną przewagę po stronie ludzi z mniejszym dochodem, można się tego spodziewać. Oczywiście jest widoczny ogon, który pokrywa tę niewielką część ludzi bogatych.

Oszczędności - okazuje się, że większość ludzi ma małe oszczędności. Podobnie jak przy dochodzie widoczny jest ogon.

Wydatki żywieniowe - zaskakującym jest, że większość gospodarstw ma tak duże wydatki, podczas gdy można by się spodziewać wykresu normalnego

Zadanie 2

In [ ]:
# Wybór wybranych kolumn
selected_columns = ['waga', 'wzrost', 'dzieci', 'wiek', 'dochod', 'oszczednosci', 'wydatki_zyw']
selected_df = df[selected_columns]

# Obliczenie macierzy korelacji dla wybranych kolumn
correlation_matrix = selected_df.corr()

# Wyświetlenie mapy ciepła
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", vmin=-1, vmax=1)
plt.title('Mapa ciepła współczynnika korelacji między zmiennymi ilościowymi')
plt.show()

Przyjmijmy takie widełki:

- |r| < 0.14: korelacja słaba
- 0.14 < |r| < 0.6: korelacja umiarkowana
- 0.6 < |r|: korelacja silna

Od razu widać silne korelacje między dochodem a oszczędnościami.
Korelacje umiarkowane istnieją między oszczędnościami a wydatkami żywieniowymi,
wzrostem a wagą, wiekiem a wagą, wiekiem a dochodem, wiekiem a oszczędnościami oraz wiekiem a wzrostem.
Pozostałe pary zmiennych są słabo skorelowane.

In [ ]:
from itertools import combinations

# Lista zmiennych jakościowych
categorical_variables = ['plec', 'jednoos', 'miejsce']

# Przeglądanie wszystkich unikalnych kombinacji par zmiennych jakościowych
for var1, var2 in combinations(categorical_variables, 2):
    contingency_table = pd.crosstab(df[var1], df[var2])
    chi2, p_value, _, _ = scp.stats.chi2_contingency(contingency_table)

    print(f'Zależność między zmiennymi {var1} a {var2}:')
    print(f'Wartość p dla testu chi-kwadrat: {p_value}')
    if p_value < 0.05:
        print('Istnieje istotna zależność między zmiennymi.')
    else:
        print('Nie ma istotnej zależności między zmiennymi.')
    print('\n')

P-wartości są znacznie wyższe niż 0.05, co pozwala na odrzucenie hipotezy istnienia zależności między sprawdzanymi parami danych.

Zadanie 3

In [ ]:
# Tworzenie wykresów rozrzutu dla wszystkich zmiennych ilościowych
sns.pairplot(df, x_vars=['waga', 'wzrost', 'dzieci', 'wiek', 'dochod', 'oszczednosci'], y_vars=['wydatki_zyw'])
plt.suptitle('Wykresy rozrzutu dla zmiennych ilościowych względem wydatków na życie', y=1.02)
plt.show()

Wykresy nie mówią zbyt dużo natomiast widać, że ludzie z małą ilością oszczędności sporo wydają na żywienie, co ma sens. Im kto więcej wydaje, tym mniejsze ma oszczędności i analogicznie im więcej się oszczędza, tym mniej można wydać.

Analogiczną zależność widać między dochodami a wydatkami żywieniowymi. Tym razem ludzi, którzy bardzo dużo zarabiają, generalnie sporo wydają na żywność z pewnymi wyjątkami.

Pozostałe wykresy nie wskazują na żaden widoczny trend.

In [ ]:
# Wykres rozrzutu dla wszystkich zmiennych ilościowych względem 'wydatki_zyw' (x i y ponownie zamienione)
plt.figure(figsize=(12, 8))
sns.scatterplot(data=df, x='waga', y='wydatki_zyw', color='skyblue', label='waga')
sns.scatterplot(data=df, x='wzrost', y='wydatki_zyw', color='salmon', label='wzrost')
sns.scatterplot(data=df, x='dzieci', y='wydatki_zyw', color='lightgreen', label='dzieci')
sns.scatterplot(data=df, x='wiek', y='wydatki_zyw', color='orange', label='wiek')
sns.scatterplot(data=df, x='dochod', y='wydatki_zyw', color='purple', label='dochod')
sns.scatterplot(data=df, x='oszczednosci', y='wydatki_zyw', color='gold', label='oszczednosci')
plt.xlabel('Zmienna ilościowa')
plt.ylabel('Wydatki na żywność')
plt.title('Wykresy rozrzutu dla zmiennych ilościowych względem wydatki_zyw')
plt.legend()
plt.show()

Jest to interesująca, lecz niepraktyczna wizualizacja. Na jednym wykresie oczywiście analiza nie ma sensu, gdyż zmienne ilościowe są w znacznie różnych ilościach i różnych jednostkach. Na przykład dzieci jest co najwyżej 5, natomiast dochody są w tysiącach.

In [ ]:
# Wybór zmiennej ilościowej i zmiennej jakościowej
variable = 'wydatki_zyw'
qualitative_variable = 'miejsce'

# Tworzenie wykresu pudełkowego
plt.figure(figsize=(10, 6))
sns.boxplot(data=df, x=qualitative_variable, y=variable)
plt.title(f'Wykres pudełkowy zmiennej {variable} w podziale na {qualitative_variable}')
plt.xlabel(qualitative_variable)
plt.ylabel(variable)
plt.show()

Niezależnie od miejsca w jakim się żyje, czy jest ono duże, czy małe ze względu na liczbę mieszkańców, wydatki żywieniowe są bardzo zbliżone. Można postawić hipotezę, że zaludnienie miejsca zamieszkania nie wpływa na wydatki związane z żywieniem.

In [ ]:
# Tworzenie wykresu słupkowego skumulowanego
plt.figure(figsize=(8, 6))
sns.countplot(data=df, x='plec', hue='jednoos', palette='Set2')
plt.title('Wykres słupkowy dla płci i prowadzenia jednoosobowego gospodarstwa domowego')
plt.xlabel('Płeć')
plt.ylabel('Liczba respondentów')
plt.legend(title='Jednoosobowe gospodarstwo', loc='upper right')
plt.show()

Większość ludzi żyje w gospodarstwach wieloosobowych.
Widać także, że większy odsetek kobiet w stosunku do mężczyzn żyje w gospodarstwach wieloosobowych.

In [ ]:
# Tworzenie wykresu skrzypcowego
plt.figure(figsize=(10, 6))
sns.violinplot(data=df, x=qualitative_variable, y=variable)
plt.title(f'Wykres skrzypcowy zmiennej {variable} w podziale na {qualitative_variable}')
plt.xlabel(qualitative_variable)
plt.ylabel(variable)
plt.show()

Ciekawym wykresem może być wykres skrzypcowy wydatków żywnościowych w zależności od miejsca. Pokazuje to, co wykres pudełkowy, a także kształt rozkładu.

Zadanie 4

In [ ]:
# Średnia i odchylenie standardowe
mean_age = np.mean(df['wiek'])
std_dev_age = np.std(df['wiek'], ddof=1)
quantiles = df['wiek'].quantile([0.25, 0.50, 0.75])
print(f'Średnia: {mean_age}, Odchylenie standardowe: {std_dev_age}, Kwartyle: {quantiles}')

In [ ]:
n = len(df['wiek'])

# Poziom ufności
# 1 - alpha = 0.99 ==> alpha = 0.01
alpha = 0.01
confidence_level = 1 - alpha/2

In [ ]:
# Obliczenie przedziału ufności dla średniej
lower_bound_mean, upper_bound_mean = scp.stats.t.interval(confidence_level, df=n-1, loc=mean_age, scale=std_dev_age/np.sqrt(n))

# Obliczenie przedziału ufności dla odchylenia standardowego
lower_bound_std_dev, upper_bound_std_dev = scp.stats.t.interval(confidence_level, df=n-1, loc=0, scale=std_dev_age)

print(f'Przedział ufności dla średniej wieku: ({lower_bound_mean}, {upper_bound_mean})')
print(f'Przedział ufności dla odchylenia standardowego wieku: ({lower_bound_std_dev}, {upper_bound_std_dev})')

In [ ]:
# Dane
wiek = df['wiek']

# Obliczenie średniej i odchylenia standardowego
mean = np.mean(wiek)
std_dev = np.std(wiek, ddof=1)  # Uwzględniamy stopień swobody

# Liczba obserwacji
n = len(wiek)

# Poziom ufności
alpha = 0.01
confidence_level = 1 - alpha / 2

# Obliczenie przedziałów ufności dla średniej
ci_mean = scp.stats.norm.interval(confidence_level, loc=mean, scale=std_dev / np.sqrt(n))

# Obliczenie przedziałów ufności dla odchylenia standardowego
ci_std_dev_lower = std_dev * np.sqrt((n - 1) / scp.stats.chi2.ppf(confidence_level, df=n - 1))
ci_std_dev_upper = std_dev * np.sqrt((n - 1) / scp.stats.chi2.ppf(1 - confidence_level, df=n - 1))
ci_std_dev = (ci_std_dev_lower, ci_std_dev_upper)

# Wyświetlenie wyników
print("Przedziały ufności dla średniej wieku:", ci_mean)
print("Przedziały ufności dla odchylenia standardowego wieku:", ci_std_dev)

n = len(wiek)  # Liczba obserwacji

# Obliczenie kwartyli
q1, q2, q3 = np.percentile(wiek, [25, 50, 75])

# Obliczenie wartości z_alpha (kwantyla rozkładu normalnego)
z_alpha = scp.stats.norm.ppf(1 - alpha/2)

# Obliczenie przedziałów ufności dla każdego kwartylu
lower_bound_q1 = q1 - (z_alpha * (q3 - q1) / (2 * np.sqrt(n)))
upper_bound_q1 = q1 + (z_alpha * (q3 - q1) / (2 * np.sqrt(n)))

lower_bound_q2 = q2 - (z_alpha * (q2 - q1) / (2 * np.sqrt(n)))
upper_bound_q2 = q2 + (z_alpha * (q2 - q1) / (2 * np.sqrt(n)))

lower_bound_q3 = q3 - (z_alpha * (q3 - q2) / (2 * np.sqrt(n)))
upper_bound_q3 = q3 + (z_alpha * (q3 - q2) / (2 * np.sqrt(n)))

# Wyświetlenie wyników
print("Przedział ufności dla pierwszego kwartyla:", (lower_bound_q1, upper_bound_q1))
print("Przedział ufności dla drugiego kwartyla:", (lower_bound_q2, upper_bound_q2))
print("Przedział ufności dla trzeciego kwartyla:", (lower_bound_q3, upper_bound_q3))

Zakładamy, że zmienna wiek ma rozkład normalny, co możemy zrobić bazując na wykresie z zadania pierwszego. Bez tego założenia, a raczej z założeniem przeciwnym o braku rozkładu normalnego użycie funkcji norm.ppf nie miałoby sensu.

Zadanie 5

In [ ]:
quantiles = df['dochod'].quantile([0.25, 0.75, 0.90])
df.loc[:, 'klasa_zamożności'] = pd.cut(df['dochod'], bins=[-np.inf, quantiles.iloc[0], quantiles.iloc[1],
                                       quantiles.iloc[2], np.inf], labels=['klasa_niższa', 'klasa_średnia', 'klasa_wyższa_średnia', 'klasa_wyższa'])

In [ ]:
plt.figure(figsize=(10, 6))
sns.boxplot(x='klasa_zamożności', y='wydatki_zyw', data=df)
plt.title('Wydatki na żywność według klasy zamożności')
plt.xlabel('Klasa zamożności')
plt.ylabel('Wydatki na żywność')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Grupowanie danych według klasy zamożności i obliczenie odchylenia standardowego dla wydatków na żywność
std_dev_by_class = df.groupby('klasa_zamożności')['wydatki_zyw'].std()

# Tworzenie wykresu odchylenia standardowego dla klas zamożności
plt.figure(figsize=(10, 6))
std_dev_by_class.plot(kind='bar', color='skyblue')
plt.title('Odchylenie standardowe wydatków na żywność według klasy zamożności')
plt.xlabel('Klasa zamożności')
plt.ylabel('Odchylenie standardowe')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

Najmniej zróżnicowane są wydatki w klasie niższej, następnie w wyższej, średniej i najbardziej w klasie wyższej średniej.
Co więcej odchylenie standardowe w klasie niższej jest ponad dwukrotnie niższe niż w klasie wyższej średniej.

Co ciekawe, wydatki żywieniowe klasy niższej są wyższe od wydatków klasy wyższej średniej, a średnie wydatki są najwyższe spośród wszystkich klas.

Zadanie 6

In [ ]:
kobiety = df[df['plec'] == 1]['oszczednosci']
mezczyzni = df[df['plec'] == 2]['oszczednosci']

In [ ]:
# Liczba kobiet
liczba_kobiet = df[df['plec'] == 1]['plec'].count()

# Liczba mężczyzn
liczba_mezczyzn = df[df['plec'] == 2]['plec'].count()

print("Liczba kobiet:", liczba_kobiet)
print("Liczba mężczyzn:", liczba_mezczyzn)

In [ ]:
# Obliczenie wariancji dla kobiet
var_kobiety = kobiety.var()

# Obliczenie wariancji dla mężczyzn
var_mezczyzni = mezczyzni.var()

# Obliczenie ilorazu wariancji
iloraz_wariancji = var_kobiety / var_mezczyzni

print("Iloraz wariancji:", iloraz_wariancji)

In [ ]:
# Przeprowadzenie testu U Manna-Whitneya
U_statistic, p_value = scp.stats.mannwhitneyu(kobiety, mezczyzni, alternative='less')

print("Wartość statystyki U (Mann-Whitney):", U_statistic)
print("Wartość p:", p_value)

Hipoteza zerowa: kobiety mają niemniejsze oszczędności w stosunku do mężczyzn.

Hipoteza alternatywna: kobiety mają mniejsze oszczędności niż mężczyźni.

Jako test statystyczny wybrałem test Manna-Whitney'a. Jest to popularny test do sprawdzania rzeczy tego typu i nasze dane spełniają jego założenia.

To badanie nie potwierdza w żaden sposób, aby kobiety miały większe oszczędności niż mężczyźni.

In [ ]:
# Obliczenie korelacji Pearsona między proporcją wydatków na żywność a oszczędnościami
korelacja, p_value = scp.stats.pearsonr(df['wydatki_zyw'] / df['dochod'], df['oszczednosci'])

print("Współczynnik korelacji Pearsona:", korelacja)
print("Wartość p:", p_value)

# Obliczenie korelacji Spearmana między proporcją wydatków na żywność a oszczędnościami
korelacja_spearmana, p_value_spearman = scp.stats.spearmanr(df['wydatki_zyw'] / df['dochod'], df['oszczednosci'])

print("Współczynnik korelacji Spearmana:", korelacja_spearmana)
print("Wartość p dla korelacji Spearmana:", p_value_spearman)

Hipoteza zerowa: niższa proporcja wydatków na żywność względem dochodu jest skorelowana z wyższymi oszczędnościami

Hipoteza alternatywna: niższa proporcja wydatków na żywność względem dochodu nie jest skorelowana z wyższymi oszczędnościami

Standardowo wybrałem korelacje Pearsona, ale dla pewności sprawdziłem też korelację Spearmana.

Warotść p jest bardzo mała, więc nie odrzucamy hipotezy o istnieniu korelacji.

Wartość korelacji jest bardzo silna ujemna i jest to oczywiste, gdyż im więcej oszczędzamy, tym mniej możemy wydać na żywność.

In [ ]:
# Definicja próby referencyjnej
sample_reference = np.repeat(56, len(df[df['plec'] == 1]))

# Przeprowadzenie testu 1samp
t_stat, p_value = scp.stats.ttest_1samp(df[df['plec'] == 1]['waga'], 56, alternative='greater')

print("Wartość statystyki t:", t_stat)
print("Wartość p dla testu 1samp:", p_value)

Hipoteza zerowa: średnia waga kobiet w próbie jest równa 56 kg

Hipoteza alternatywna: średnia waga kobiet w próbie jest wyższa niż 56 kg

Można użyć testu 1samp.

Wartość p jest bardzo mała, więc średnia waga prawdopodobnie jest wyższa niż 56kg.

In [ ]:
# Określenie wartości referencyjnej
# Dla rozkładu Bernoulliego, wartość referencyjna jest często ustawiana na 0.5
wartosc_referencyjna = 0.5

# Przeprowadzenie jednostronnego t-testu
t_stat, p_value = scp.stats.ttest_1samp(df['jednoos'], wartosc_referencyjna)

print("Wartość statystyki t:", t_stat)
print("Wartość p dla testu t:", p_value)

Hipoteza zerowa: zmienna 'jednoos' ma rozkład bernoulliego.

Hipoteza alternatywna: zmienna 'jednoos' nie ma rozkładu bernoulliego.

Hipoteza nie została obalona przez wykonany test 1samp.

Zadanie 7

In [ ]:
df = df_original.copy()
df = df[df['wydatki_zyw'] >= 0]

# Usunięcie kolumny 'id'
df.drop('id', axis=1, inplace=True)

# Dodanie stałej do modelu
df = sm.add_constant(df)

# Kodowanie zmiennych jakościowych
df = pd.get_dummies(df, columns=['plec', 'jednoos', 'miejsce'])

df

In [ ]:
# Definicja zmiennych objaśniających i objaśnianej
X = df.drop('wydatki_zyw', axis=1)
y = df['wydatki_zyw']

# Wstępne dopasowanie modelu
model = sm.OLS(y.astype(float), X.astype(float)).fit()
model.summary()

In [ ]:
ftest = model.f_test(np.identity(len(model.params)))
print(ftest)

# ttest jest też w skrócie modelu powyżej
ttest = model.t_test(np.identity(len(model.params)))
print(ttest)

Przebadane zależności tłumaczą niecałe 68% zmiennych zależnych (R^2 = 0.679). Reszta pozostaje dla nas tajemnicą.

Test F wykazał bardzo istotny związek między zmiennymi niezależnymi a zmienną zależną (F=1465.56, p<0.001). Stopnie swobody to odpowiednio 293 dla mianownika i 11 dla licznika testu F. Wynik ten wskazuje na dużą siłę efektu zmiennych niezależnych na zmienną zależną.

W przedstawionych wynikach t-testu, każdy wiersz odpowiada jednej zmiennej niezależnej. Wartość "coef" oznacza oszacowaną wartość współczynnika regresji dla danej zmiennej niezależnej. Kolumna "std err" przedstawia błąd standardowy estymacji współczynnika. Wartość "t" to wartość t-statystyki, która mierzy, jak bardzo odstająca jest nasza wartość oszacowania od zera, przy założeniu, że nie ma efektu (tj. wartość współczynnika wynosi zero). P>|t| to wartość p, która informuje nas o istotności statystycznej estymowanej wartości współczynnika; im niższa wartość p, tym bardziej istotny jest współczynnik. Przedział ufności 95% dla każdego współczynnika znajduje się w kolumnach [0.025, 0.975].

Widzimy także, że czynnniki z niską p-wartością to dochód, oszczędności, bycie kobietą, życie w dużym mieście i mieszkanie w gospodarstwie jednoosobowym. Są to zmienne indywidualnie istotne.

In [ ]:
print(smd.linear_reset(model, power = 2, test_type = "fitted"))

In [ ]:
# Obliczenie statystyki testowej i wartości p dla testu Breuscha-Pagana
test_statistic, p_value, _, _ = smd.het_breuschpagan(model.resid, X)

# Interpretacja wyników
print("Statystyka testowa Breuscha-Pagana:", test_statistic)
print("Wartość p dla testu Breuscha-Pagana:", p_value)

if p_value > 0.05:
    print("Nie ma dowodów na heteroskedastyczność.")
else:
    print("Dane wskazują na heteroskedastyczność.")

In [ ]:
dw_statistic = durbin_watson(model.resid)

print("Wartość statystyki testu Durbin-Watsona:", dw_statistic)

In [ ]:
# Test Shapiro-Wilka
stat, p_value = scp.stats.shapiro(model.resid)

# Wyświetlenie wyników testu
print("Statystyka testu Shapiro-Wilka:", stat)
print("Wartość p:", p_value)

# Interpretacja wyników
alpha = 0.05
if p_value > alpha:
    print("Reszty mogą mieć rozkład normalny.")
else:
    print("Reszty mogą nie mieć rozkładu normalnego.")

Na podstawie testu RESET możemy skłaniać się ku odrzuceniu liniowej formy modelu (nie możemy odrzucić nieliniowości).

Test Breuscha-Pagana niestety nie odrzuca heteroskedastyczności.

Nie ma jednoznacznych dowodów na autokorelację.

Reszty nie mają rozkładu normalnego.

In [ ]:
# Obliczenie współczynników VIF dla wszystkich zmiennych objaśniających
vif = pd.DataFrame()
vif["Features"] = X.columns
vif["VIF"] = [variance_inflation_factor(X.values.astype(float), i) for i in range(X.shape[1])]
print(vif)

"plec_1", "plec_2", "jednoos_0", "jednoos_1", "miejsce_1", "miejsce_2" i "miejsce_3" dają \infty, co sugeruje, że istnieje perfekcyjna współliniowość między tymi zmiennymi a innymi zmiennymi w modelu.

W przypadku zmiennych "dochod" i "oszczednosci" wartości VIF są znacznie wyższe niż 10, co sugeruje obecność silnej współliniowości między nimi a innymi zmiennymi objaśniającymi.

Już na samym początku usunąłem niepoprawne dane.

In [ ]:
df = df_original.copy()
df = df[df['wydatki_zyw'] >= 0]

# Usunięcie kolumny 'id'
df.drop('id', axis=1, inplace=True)

# Dodanie stałej do modelu
df = sm.add_constant(df)

# Kodowanie zmiennych jakościowych
df = pd.get_dummies(df, columns=['plec', 'jednoos', 'miejsce'])

# Dodanie nowych zmiennych
df['dochod_oszczednosci'] = df['dochod'] - df['oszczednosci']
df['dochod_oszczednosci_squared'] = df['dochod_oszczednosci'] ** 2

df.drop(['dochod', 'oszczednosci'], axis=1, inplace=True)

# Definicja zmiennych objaśniających i objaśnianej
X = df.drop('wydatki_zyw', axis=1)
y = df['wydatki_zyw']

# Wstępne dopasowanie modelu
model = sm.OLS(y.astype(float), X.astype(float)).fit()
model.summary()

VIF pokazał, że oszczędności i dochody mają problemy, więc metodą prób i błędów doszedłem do zmiany ich na ich różnicę oraz jej kwadrat.

In [ ]:
print(smd.linear_reset(model, power = 2, test_type = "fitted"))
# Obliczenie statystyki testowej i wartości p dla testu Breuscha-Pagana
test_statistic, p_value, _, _ = smd.het_breuschpagan(model.resid, X)

# Interpretacja wyników
print("Statystyka testowa Breuscha-Pagana:", test_statistic)
print("Wartość p dla testu Breuscha-Pagana:", p_value)

if p_value > 0.05:
    print("Nie ma dowodów na heteroskedastyczność.")
else:
    print("Dane wskazują na heteroskedastyczność.")
dw_statistic = durbin_watson(model.resid)

print("Wartość statystyki testu Durbin-Watsona:", dw_statistic)
# Test Shapiro-Wilka
stat, p_value = scp.stats.shapiro(model.resid)

# Wyświetlenie wyników testu
print("Statystyka testu Shapiro-Wilka:", stat)
print("Wartość p:", p_value)

# Interpretacja wyników
alpha = 0.05
if p_value > alpha:
    print("Reszty mają rozkład normalny.")
else:
    print("Reszty nie mają rozkładu normalnego.")

Nie ma dowodów żeby model nie spełniał warunków KMRL.

In [ ]:
# Indeks współczynnika dla zmiennej waga i wzrost
index_waga = X.columns.get_loc('waga')
index_wiek = X.columns.get_loc('wiek')

# Współczynniki beta dla wagi i wzrostu
beta_waga = model.params[index_waga]
beta_wiek = model.params[index_wiek]

print("Waga:")
print(f"Każdy dodatkowy kilogram wagi jest związany ze wzrostem wydatków na żywność o {beta_waga:.2f} jednostek.")

print("\nWiek:")
print(f"Każdy dodatkowy rok życia jest związany ze wzrostem wydatków na żywność o {beta_wiek:.2f} jednostek.")

Wybrałem wagę i wiek, ponieważ sądząc po bardzo niskim P, można sądzić, że są indywidualnie istotne.

In [ ]:
# Utwórz nową kolumnę w ramce danych zawierającą przewidywane wydatki na żywność
df['przewidywane_wydatki_zyw'] = model.predict(X)

# Oblicz kwantyl 90% przewidywanych wydatków na żywność
kwantyl_90 = df['przewidywane_wydatki_zyw'].quantile(0.90)

# Wybierz osoby, których przewidywane wydatki na żywność przekraczają kwantyl 90%
osoby_gorne_10_proc = df[df['przewidywane_wydatki_zyw'] > kwantyl_90]

# Wyświetl opisowe statystyki dla tych osób
opisowe_statystyki = osoby_gorne_10_proc.describe()

print(opisowe_statystyki)

Ludzie w tej grupie mają znacznie więcej dzieci niż średnio. Widać to poprzez porównanie otrzymanych wartości z wartościami z zadania pierwszego. Ma to sens, gdyż dzieci potrzebują jedzenia, więc im więcej się ma dzieci, tym większe powinny być wydatki na żywność.

Waga, wzrost i wiek są podobne do średniej.

Wydaje się też, że dochody w tej grupie osób są wyższe, ale jest to trudne do porównania ze względu na zmianę zmiennych.